In [ ]:
import numpy as np
from scipy import stats as spstats
from scipy.stats import moment, kurtosis, skew, mode, zscore, pearsonr
from scipy.optimize import fsolve, root
import dask.array as da
from matplotlib import pyplot as plt
import sympy as sym
from sympy import exp, symbols, lambdify

In [ ]:
memmap_array = np.lib.format.open_memmap('matrix_all.npy', mode='r')
memmap_array_time = np.lib.format.open_memmap('t_all.npy', mode='r')
matrix_C_XXX=np.load("ckk_all.npy")
L6=np.load("L6_regions2channels_abs_sum.npy")

# Create a Dask array from the memory-mapped NumPy array
# Adjust 'chunks' based on your available system memory and desired chunk size
chunks_size = (400, 528, 10000) 
chunks_size_time = (10000) 
dask_array = da.from_array(memmap_array, chunks=chunks_size)
dask_array_time = da.from_array(memmap_array_time, chunks=chunks_size_time)

In [ ]:
time_trans=2000
signals_rate_0=(dask_array[:,0:88,time_trans:]).compute()
signals_rate=signals_rate_0.T
print(f"Dimension signals_rate (combinations,nodes,time):", signals_rate_0.shape)
V_0=(dask_array[:,88:176,time_trans:]).compute()
V=V_0.T
print(f"Dimension V (combinations,nodes,time):", V_0.shape)
t_all=(dask_array_time[time_trans:]).compute()
DP_0=(dask_array[:,440:528,time_trans:]).compute()
Dp=DP_0.T
print(f"Dimension [Dp] (combinations,nodes,time):", DP_0.shape)
ckk_inh=matrix_C_XXX[0][0]
ckk_exc=matrix_C_XXX[0][1]
ckk_dopa_min=np.min(matrix_C_XXX[:,2])
ckk_dopa_max=np.max(matrix_C_XXX[:,2])
print(f"min_combination:{ckk_inh},{ckk_exc},{ckk_dopa_min}")
print(f"max_combination:{ckk_inh},{ckk_exc},{ckk_dopa_max}")

In [ ]:
mean_DP_over_regions = np.mean(Dp[:,:,:],axis=1) 
std_DP = np.std(Dp[:,:,:],axis=1, ddof=1)  # ddof=1 for sample standard deviation
print(f"the mean Dp concentration of all network. Shape of vector:",{mean_DP_over_regions.shape})
mean_DP_over_time = np.mean(Dp[:,:,:],axis=0)  
std_DP_regions = np.std(Dp[:,:,:],axis=0, ddof=1)  # ddof=1 for sample standard deviation

DP=np.mean(mean_DP_over_time,axis=0)
std_DP=np.std(mean_DP_over_time,axis=0)

# functions to compute avalanches

In [ ]:
# avalanches_global_pattern(x, n)
# f is a matrix (containing the consecutive series of activation of one patients (by consecutive I mean that all the clean bits have been attached one after the other)
# of axb, where a is the number of areas and b is the time. The argument n will specify how many regions to take into account. h will provide y vectors, with y the number
# of avalanches, each vector being long n, that is full with logicals where 0 means that that particular regions has never been active in that
# particular avalanches, and 1 means the opposite. If a region has been active only in 1 time bin or in more than 1 time bin, it will be 1 in the vector eitherway.

def avalanches_global_pattern(x, n):
    x = x[:n, :]
    activations_bins = np.where(np.any(x, axis=0))[0]
    activations_bins = np.concatenate((activations_bins, [5, 6]))  # Adding dummy values at the end
    gg = np.diff(activations_bins) != 1
    Index_start = np.where(gg == 1)[0]

    f = []
    h = []
    f.append(x[:, activations_bins[0]:activations_bins[Index_start[0]]+1])

    for ii in range(len(Index_start) - 1):
        f.append(x[:, activations_bins[Index_start[ii]+1]:activations_bins[Index_start[ii + 1]]+1]) 

    for kk in range(len(f)):
        a = np.sum(f[kk], axis=1)
        h.append(a)
    return f, h

In [ ]:
# func_transition_matrix
# input: binary matrix b representing active region, structured as regions x %times. Out is an asymmetric matrix, called transition matrix, 
# containing the probability that, if not i is active, node j will be active in the next timestep

def func_transition_matrix(b):
    num_regions, num_times = b.shape
    out = np.zeros((num_regions, num_regions))
    
    num_activations_per_reg = np.sum(b, axis=1)
    
    for kk1 in range(num_regions):
        g_loop = np.zeros(num_regions)
        for kk2 in range(num_times - 1):
            if b[kk1, kk2] == 1:
                curr_reg = np.repeat(b[kk1, kk2], num_regions)
                next_reg = b[:, kk2 + 1]
                g = curr_reg == next_reg
                g_loop += g
        g_loop[g_loop != 0] /= num_activations_per_reg[kk1]
        out[kk1, :] = g_loop
    
    return out

In [ ]:
# compute_ATM(b1)
# b1 is the binarized matrix.
# num_cc is the number of conditions or channels.
# output: ATM and number of avalanches (num_avalan) for each channel or condition.

def compute_ATM(b1):
    num_avalan = np.zeros(b1.shape[2])
    ttt_noaval_accumulated = []
    for cc in range(b1.shape[2]):
        (aval_nobin,h) = avalanches_global_pattern(b1[:,:,cc], b1.shape[0])

        num_avalanche = np.zeros(len(aval_nobin))
        temp_TMs_nobin = np.zeros((b1.shape[0], b1.shape[0], len(aval_nobin)))

        for kk_aval in range(len(aval_nobin)):
            num_avalanche[kk_aval] = aval_nobin[kk_aval].shape[1]
            tt_nobin = func_transition_matrix(aval_nobin[kk_aval])
            temp_TMs_nobin[:, :, kk_aval] = (tt_nobin + tt_nobin.T) / 2

        num_avalan[cc] = np.mean(num_avalanche)

        ttt_noaval = np.sum(temp_TMs_nobin, axis=2) / np.sum(temp_TMs_nobin != 0, axis=2)
        ttt_noaval[np.isnan(ttt_noaval)] = 0
        ttt_noaval_accumulated.append(ttt_noaval)
        
    return num_avalan, np.array(ttt_noaval_accumulated)

In [ ]:
# compute LL
import numpy as np
from scipy.stats import zscore

def compute_LL(V, L6):
    V = np.transpose(V, (2, 1, 0)) #V ~ combinations x regions x time
    print(f"Shape of V (comb,regions,time) ={V.shape}")
    rr1 = V[:, 0:34, :]
    rr2 = V[:, 52:86, :]
    reduced_V = np.concatenate((rr1, rr2), axis=1)
    
    # Step 3: Compute regions2channel64_nozscored and regions2channel6_nozscored
    regions2channel6_nozscored = np.dot(L6, reduced_V)
    
    # Step 4: Extract STN time series and reshape
    STN_lh_time_series_nozscored = V[:, 42, :].reshape(1, -1, V.shape[2])
    STN_rh_time_series_nozscored = V[:, 45, :].reshape(1, -1, V.shape[2])
    Final_matrix_6_nozscored = np.concatenate((regions2channel6_nozscored, STN_lh_time_series_nozscored,STN_rh_time_series_nozscored), axis=0)
    
    # Step 6: Z-score normalization
    Final_matrix_6 = np.array([zscore(matrix, axis=1) for matrix in Final_matrix_6_nozscored])
    
    # Step 7: Transpose matrices
    LL_6 = np.transpose(Final_matrix_6, (0, 2, 1))
    
    return LL_6

# Analysis

In [ ]:
mean_over_regions = np.mean(signals_rate[:,:,:],axis=1) 
std = np.std(signals_rate[:,:,:],axis=1)  
mean_over_time = np.mean(signals_rate[:,:,:],axis=0)  
std_regions = np.std(signals_rate[:,:,:],axis=0) 

rr=np.mean(mean_over_time,axis=0)
std=np.std(mean_over_time,axis=0)
rr2=np.mean(mean_over_regions,axis=0)
std2=np.std(mean_over_regions,axis=0)

In [ ]:
meanV_over_regions = np.mean(V[:,:,:],axis=1) 
stdV = np.std(V[:,:,:],axis=1)  
print(f"the mean V of all network. Shape of vector: {meanV_over_regions.shape} and the min and max value are: {np.min(meanV_over_regions)},{np.max(meanV_over_regions)}")
meanV_over_time = np.mean(V[:,:,:],axis=0)  
stdV_regions = np.std(V[:,:,:],axis=0) 
print(f"the mean V of each node. Shape of vector: {meanV_over_time.shape} and the min and max value are: {np.min(meanV_over_time)},{np.max(meanV_over_time)}")

VV=np.mean(meanV_over_time,axis=0)
stdV=np.std(meanV_over_time,axis=0)
print(f"the mean V over all the nodes for each cc. Shape of vector: {VV.shape} and the min and max value are: {np.min(VV)},{np.max(VV)}")
VV2=np.mean(meanV_over_regions,axis=0)
std2V=np.std(meanV_over_regions,axis=0)
print(f"the mean V over all time for each cc. Shape of vector: {VV2.shape} and the min and max value are: {np.min(VV2)},{np.max(VV2)}")

In [ ]:
cc_idx=[352,43,62] # if you want to chose only some combinations

In [ ]:
# plot for rate time series and fast variables plane
%matplotlib widget
num_comb=len(cc_idx)
fig, axes = plt.subplots(nrows=num_comb, ncols=1, figsize=(6, 5),sharex='col')  # Share X-axis across subplots

for i, cc in enumerate(cc_idx):
    ax = axes[i]
    for region in range(signals_rate.shape[1]):
        ax.plot(signals_rate[4000:, region, cc],alpha=0.9)
    
    ax.set_ylabel('r',fontsize=20)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    
    # Add a legend indicating the specific value
    c_dopa = matrix_C_XXX[cc][2]
    legend_text = f'$w_{{dopa}}$ = {c_dopa:.2f}'  
    handle, = ax.plot([], [], label=legend_text)
    
    if i == num_comb - 1:
        ax.set_xlabel('Time (ms)', fontsize=20)
        ax.spines['bottom'].set_visible(True)
    else:
        ax.tick_params(labelbottom=False)  
    
    ax.set_xlim(0, 4000)

plt.subplots_adjust(hspace=0.01)  # Adjust hspace according to your needs
plt.tight_layout()

In [ ]:
threshold =2
binarized_matrix = np.where(signals_rate_zscored > threshold, 1, 0)
print(f"The dimension of binarized_matrix is {binarized_matrix.shape}")
activations = binarized_matrix.sum(axis=0)
print(f"For each combination, compute the number of activations for each region over time, so the dimension of mean_activations is {activations.shape}")

# Lead field matrix

In [ ]:
LL_6 = compute_LL(V, L6)
print(f"LL_6 shape: {LL_6.shape}")

In [ ]:
threshold = 2.3
binarized_matrix = np.where(LL_6> threshold, 1, 0)
(num_avalan, ttt_noaval)=compute_ATM(binarized_matrix)
print(f"The dimension of binarized_matrix is {binarized_matrix.shape}")
print(f"for each cc combination we have an ATM, so in this case we have {len(ttt_noaval)} matrices")
print(f"Example for one combination _ regions x regions x num_avalanches: {ttt_noaval[0].shape}")

# compute the data features

In [ ]:
#def calculate_summary_statistics_on_ATM(matrixATM):

from scipy.linalg import eig, det
from numpy.linalg import norm
from sklearn.decomposition import PCA
from scipy.stats import hmean

def calculate_summary_statistics_on_ATM(matrixATM):
    summary_statistics = []
    mean_ATM = np.zeros(matrixATM.shape[0])
    std_ATM = np.zeros(matrixATM.shape[0])
    median_ATM = np.zeros(matrixATM.shape[0])
    sum_ATM = np.zeros(matrixATM.shape[0])
    skw_ATM = np.zeros(matrixATM.shape[0])
    kurt_ATM = np.zeros(matrixATM.shape[0])
    max_eigen_val_ATM = np.zeros(matrixATM.shape[0])
    trace_ATM = np.zeros(matrixATM.shape[0])
    std_diag_ATM = np.zeros(matrixATM.shape[0])
    CV_ATM = np.zeros(matrixATM.shape[0])
    mom1_ATM = np.zeros(matrixATM.shape[0])
    mean_over_kurtosis= np.zeros(matrixATM.shape[0])
    kurt_diag= np.zeros(matrixATM.shape[0])
    norm_ATM = np.zeros(matrixATM.shape[0])
    harmonic_mean_ATM = np.zeros(matrixATM.shape[0])
    entropy_ATM=np.zeros(matrixATM.shape[0])
    
    for cc in range(matrixATM.shape[0]):
        atm = matrixATM[cc, :, :]
        #mean_ATM[cc] = np.mean(atm)                     # Compute mean
        mean_ATM[cc] = np.log(np.mean(atm))                     # Compute mean
        std_ATM[cc] = np.std(atm, ddof=1)               # Compute standard deviation
        nonzero_elements = atm[atm != 0]                # Compute median
        #median_ATM[cc] = np.median(nonzero_elements)
        median_ATM[cc] = np.log(np.median(nonzero_elements))
        #sum_ATM[cc] = np.sum(atm)                       # Compute sum
        sum_ATM[cc] = np.log(np.sum(atm))                       # Compute sum
        harmonic_mean_ATM[cc] = hmean(atm.ravel())
        #skw_ATM[cc] = skew(atm.flatten())               # Compute skewness
        skw_ATM[cc] = np.exp(skew(atm.flatten()))               # Compute skewness

        kurt_ATM[cc] = kurtosis(atm.flatten())          # Compute kurtosis
        mean_over_kurtosis[cc]=np.mean(atm)/kurtosis(atm.flatten())
        #trace_ATM[cc] = np.trace(atm)                   # Compute trace
        trace_ATM[cc] = np.exp(np.trace(atm))                   # Compute trace
        diag_elements = np.diag(atm)                    # Extract diagonal elements
        std_diag_ATM[cc] = np.std(diag_elements, ddof=1) # Compute standard deviation of diagonal elements
        kurt_diag[cc]=kurtosis(diag_elements)                    # Extract diagonal elements
        norm_ATM[cc]=np.linalg.norm(atm)
        # mom1_ATM[cc]= np.log10(np.mean(atm)/np.std(atm))  
        # CV_ATM[cc]= np.log10(np.std(atm)/np.mean(atm))
        mom1_ATM[cc]= (np.mean(atm)/np.std(atm))  
        CV_ATM[cc]= (np.std(atm)/np.mean(atm))
        #CV_ATM[cc]= np.mean(atm)/np.std(atm, ddof=1)
        entropy_ATM[cc]=matrix_entropy(atm, bins='auto', norm=True)
                
        if atm.shape[0] == atm.shape[1]:
            eigen_vals_ATM, _ = eig(atm)  
            #max_eigen_val_ATM[cc] = np.max(eigen_vals_ATM)
            max_eigen_val_ATM[cc] = np.exp(np.max(eigen_vals_ATM))    

    summary_statistics.append([
            mean_ATM, std_ATM, median_ATM, sum_ATM, skw_ATM, kurt_ATM,
            max_eigen_val_ATM,trace_ATM, std_diag_ATM,CV_ATM,mom1_ATM, mean_over_kurtosis,kurt_diag,norm_ATM,
            harmonic_mean_ATM,entropy_ATM
        ])

    return np.array(summary_statistics)

In [ ]:
def entropy(p, tol=1e-8):
    """Calculate the entropy of a probability distribution, ensuring no zero probability values."""
    p = np.clip(p, tol, 1 - tol)  # Prevent division by zero or log of zero
    return -np.sum(p * np.log2(p))

def matrix_entropy(A, bins=5, norm=True):
    """Calculate the entropy of the matrix A using histogram bins."""
    A = A.flatten() 
    # Generate histogram
    try:
        p, bin_edges = np.histogram(A, bins=bins)
    except Exception as e:
        # Catch all exceptions to understand the issue better
        raise Exception(f"Failed to generate histogram with `bins`={bins}: {str(e)}")

    if p.sum() == 0:
        # If the histogram is empty (all bins are zero)
        H = 0
    else:
        p = p / p.sum()
        H = entropy(p)  # Calculate entropy

    # Check if normalization of entropy is needed
    n_bins = len(bin_edges) - 1  # Number of bins is number of edges minus one
    if norm and n_bins > 1: 
        H /= np.log2(n_bins)
    return H


In [ ]:
# calculate_summary_statistics_on_V
def calculate_summary_statistics_V(LL):
    summary_statistics_V = []
    mean_FC = np.zeros(LL.shape[2])
    mean_kurtosis = np.zeros(LL.shape[2])
    mean_co_kurtosis=np.zeros(LL.shape[2])
    mean_max_co_kurtosis=np.zeros(LL.shape[2])
    mean_dev_co_kurtosis=np.zeros(LL.shape[2])            
    mean_covariance=np.zeros(LL.shape[2])
    mean_cross_correlation=np.zeros(LL.shape[2])
    var_cross_correlation=np.zeros(LL.shape[2])  
    mean_corr_entropy=np.zeros(LL.shape[2]) 
    var_corr_entropy=np.zeros(LL.shape[2]) 
        
    for cc in range(LL.shape[2]):
        data = LL[:, :, cc]
        # data has the shape: regions x times
        correlation_matrix = np.corrcoef(data)
        Kurto = np.mean(np.power(data - np.mean(data, axis=1, keepdims=True), 4), axis=1) / np.power(np.var(data, axis=1), 2)
        co_Kurto = np.zeros((LL.shape[0], LL.shape[0]))
        corr_entropy = np.zeros((LL.shape[0], LL.shape[0]))

        for i in range(LL.shape[0]):
            for j in range(LL.shape[0]):
                co_Kurto[i,j] = np.mean(np.power(data[i,:] - np.mean(data[i,:], keepdims=True), 2) * np.power(data[j,:] - np.mean(data[j,:], keepdims=True), 2)) / (np.var(data[i,:]) * np.var(data[j,:]))
                entropy_i=matrix_entropy(data[i,:], bins='auto', norm=True)
                entropy_i.shape
                entropy_j=matrix_entropy(data[j,:], bins='auto', norm=True)
                corr_entropy[i,j]= np.mean((entropy_i-entropy_j))

        mean_FC[cc] = np.mean(correlation_matrix)               
        mean_kurtosis[cc] = np.mean(Kurto) 
        
        np.fill_diagonal(co_Kurto, np.nan)
        mean_co_kurtosis[cc]=np.nanmean(co_Kurto)

        mean_max_co_kurtosis[cc]=np.nanmax(co_Kurto)
        mean_dev_co_kurtosis[cc]=np.nanstd(co_Kurto)

        mean_covariance[cc] = np.mean(np.cov(data))
        var_cross_correlation[cc]=np.var(correlation_matrix) 
        
        mean_corr_entropy[cc]=np.mean(corr_entropy)
        var_corr_entropy[cc]=np.var(corr_entropy)

        summary_statistics_V.append([
            mean_FC, mean_kurtosis,mean_co_kurtosis,mean_max_co_kurtosis, mean_dev_co_kurtosis,
            mean_covariance,var_cross_correlation,mean_corr_entropy,var_corr_entropy    
        ])
        
    return np.array(summary_statistics_V)

#  compute the data features

In [ ]:
# unique function: calculate_features
def calculate_features(LL,threshold):
    binarized_matrix = np.where(LL> threshold, 1, 0)
    (num_avalan, ttt_noaval)=compute_ATM(binarized_matrix)
    summary_statistics=calculate_summary_statistics_on_ATM(ttt_noaval)
    summary_statistics_V=calculate_summary_statistics_V(LL)
    print(f"There are {summary_statistics_V.shape[1]} features calculated on signals")
    print(summary_statistics_V.shape)
    summary_statistics=np.array(summary_statistics)
    print(f"There are {summary_statistics.shape[1]} features calculated on ATM")
    summary_statistics = np.squeeze(summary_statistics)
    summary_statistics_V = summary_statistics_V[0,:,:]
    combined_statistics = np.concatenate((summary_statistics, summary_statistics_V), axis=0)
    return np.array(combined_statistics)

In [ ]:
# uploading one simulations composed by 50 different values of ckk_dopa we can have an idea of what happens 
# for low and high values of ckk_dopa and we can estimate the range of parameters. 
# Then we also computed the statistics from the data.
# the function calculate_features take in input the LL_6 matrix which is the product between the V signals simulated 
# and the Lead field matrix for each value of ckk_dopa
print(f"The shape of LL_6 is:{LL_6.shape} num_channels x time x number of ckk_dopa values ")
threshold=1
all_data_features=calculate_features(LL_6,threshold)

In [ ]:
print(f"In this way, for each of {all_data_features.shape[1]} c_dopa value, we have {all_data_features.shape[0]} data features")

In [ ]:
np.save("synthetic_data_features.npy",all_data_features)
np.save("c_dopa.npy",matrix_C_XXX[:,2])